# Implementación Path Relinking
---

In [51]:
from structure import instance, solution
from algorithms import grasp
from structure.solution import evaluate
from copy import deepcopy
import pandas as pd
import random

In [52]:
path = "instances/MDG-a_12_100_m10.txt"
inst = instance.readInstance(path)
sol, conjunto_soluciones, tiempo_final_grasp = grasp.execute(inst, 1, 0.1)

In [53]:
conjunto_soluciones_pd = pd.DataFrame(conjunto_soluciones)
print(len(conjunto_soluciones_pd))
conjunto_soluciones_pd = conjunto_soluciones_pd.drop_duplicates("sol")

750


In [54]:
conjunto_soluciones_pd.sort_values("of", ascending=False)

,instance,sol,of
28,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 96, 13, 15, 51, 20, 86, 55, 59, 93}",354.25
3,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{2, 98, 37, 90, 91, 12, 20, 26, 59, 63}",351.82
123,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18
46,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53
41,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 3, 14, 78, 15, 83, 53, 56, 61, 94}",349.45
...,...,...,...
461,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 97, 6, 75, 76, 77, 20, 21, 24, 31}",319.00
654,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{67, 70, 73, 41, 46, 49, 84, 22, 86, 30}",318.28
247,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{32, 1, 3, 4, 70, 8, 79, 84, 54, 30}",317.78
297,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 9, 43, 76, 77, 13, 21, 24, 25, 31}",317.65


In [55]:
sol_max = conjunto_soluciones_pd[conjunto_soluciones_pd["of"] == max(conjunto_soluciones_pd["of"])].iloc[0,1]
conjunto_soluciones_pd["n_distintos"] = conjunto_soluciones_pd["sol"].map(lambda x: len(x.difference(sol_max)))
conjunto_soluciones_pd[conjunto_soluciones_pd["n_distintos"] > 0].sort_values("of", ascending=False)

,instance,sol,of,n_distintos
3,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{2, 98, 37, 90, 91, 12, 20, 26, 59, 63}",351.82,8
123,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18,5
46,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53,10
41,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 3, 14, 78, 15, 83, 53, 56, 61, 94}",349.45,9
394,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{98, 2, 37, 15, 16, 20, 86, 59, 60, 93}",349.36,5
...,...,...,...,...
461,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 97, 6, 75, 76, 77, 20, 21, 24, 31}",319.00,8
654,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{67, 70, 73, 41, 46, 49, 84, 22, 86, 30}",318.28,9
247,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{32, 1, 3, 4, 70, 8, 79, 84, 54, 30}",317.78,10
297,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 9, 43, 76, 77, 13, 21, 24, 25, 31}",317.65,8


A continuación, se realiza una implementación recursiva de Path Relinking entre dos soluciones específicas.

In [56]:
sol_0 = conjunto_soluciones[14]
sol_1 = conjunto_soluciones[57]

In [58]:
def transicion(sol_0, sol_1):
    best = None
    resultado = 0
    sol_2 = deepcopy(sol_0)
    sol_guia = deepcopy(sol_1)
    sol_guia["sol"] = sol_guia["sol"].difference(sol_0["sol"])
    for s0 in sol_0["sol"]:
        for s1 in sol_guia["sol"]:
            if s1 not in sol_0["sol"]:
                sol_2["sol"].remove(s0)
                sol_2["sol"].add(s1)
                sol_2["of"] = evaluate(sol_2)
                if best is None or sol_2["of"] > best["of"]:
                    anyadido = s1
                    best = deepcopy(sol_2)
                sol_2["sol"].remove(s1)
                sol_2["sol"].add(s0)
    
    sol_guia["sol"].remove(anyadido)

    if len(sol_guia["sol"]) > 1:
        resultado = transicion(best, sol_guia)

        if resultado["of"] < best["of"]:
            resultado = best
    else:
        print(best["sol"])
        resultado = best
    

    return resultado

best = transicion(sol_0, sol_1)

{96, 2, 35, 68, 38, 47, 52, 89, 92, 29}


In [59]:
best["of"]

332.15

In [60]:
sol_0["of"]

338.37999999999994

In [61]:
sol_1["sol"]

{5, 8, 31, 35, 39, 43, 47, 62, 77, 98}

### Opción para el Path Relinking

La siguiente función es la versión final y general de PR paso a paso.

In [62]:
def relink(inicio, guia):
    current = deepcopy(inicio)
    best = deepcopy(inicio)

    to_add = guia["sol"] - current["sol"]
    to_remove = current["sol"] - guia["sol"]

    while to_add:
        vecinos = []
        for a in to_add:
            for r in to_remove:
                nuevo = deepcopy(current)
                nuevo["sol"].remove(r)
                nuevo["sol"].add(a)
                nuevo["of"] = evaluate(nuevo)
                vecinos.append(nuevo)
        if not vecinos:
            break
        current = max(vecinos, key=lambda s: s["of"])
        if current["of"] > best["of"]:
            best = deepcopy(current)

        to_add = guia["sol"] - current["sol"]
        to_remove = current["sol"] - guia["sol"]

    return best


Ahora se busca ejecutar PR entre todos los pares del conjunto élite. Mostramos cuáles son las mejores soluciones sin repetir y la mejor que encuentra.

In [ ]:
# Seleccionamos las x mejores soluciones distintas (se puede modificar el nº de soluciones que se toman)
n_soluciones = 10
elite_df = conjunto_soluciones_pd.sort_values("of", ascending=False).drop_duplicates("sol").head(n_soluciones)
elite_solutions = elite_df.to_dict("records")

# Aplicamos Path Relinking entre todos los pares de soluciones
mejor_pr = None
for i in range(len(elite_solutions)):
    for j in range(i+1, len(elite_solutions)):
        s1 = elite_solutions[i]
        s2 = elite_solutions[j]
        pr1 = relink(s1, s2)
        pr2 = relink(s2, s1)
        for cand in [pr1, pr2]:
            if mejor_pr is None or cand["of"] > mejor_pr["of"]:
                mejor_pr = cand

# Mostramos la mejor solución obtenida por Path Relinking
print("Mejor solución encontrada por Path Relinking:")
print("Conjunto:", mejor_pr["sol"])
print("Valor objetivo:", mejor_pr["of"])


Mejor solución encontrada por Path Relinking:
Conjunto: {64, 96, 13, 15, 51, 20, 86, 55, 59, 93}
Valor objetivo: 354.25


In [64]:
# Repetimos el bucle anterior, aunque esta vez para guardar todas las soluciones
soluciones_pr = []

for i in range(len(elite_solutions)):
    for j in range(i + 1, len(elite_solutions)):
        s1 = elite_solutions[i]
        s2 = elite_solutions[j]
        pr1 = relink(s1, s2)
        pr2 = relink(s2, s1)
        soluciones_pr.extend([pr1, pr2])

# Convertimos a DataFrame para análisis posterior
df_pr = pd.DataFrame(soluciones_pr).drop_duplicates(subset="sol")
print("\nTop 5 soluciones PR:")
print(df_pr.sort_values("of", ascending=False).head(5))

print(f"\nTotal de soluciones generadas mediante PR: {len(df_pr)}")


Top 5 soluciones PR:
                                             instance  \
0   {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
8   {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
14  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
18  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   

                                         sol      of  n_distintos  
0   {64, 96, 13, 15, 51, 20, 86, 55, 59, 93}  354.25            0  
8    {2, 98, 26, 37, 59, 12, 20, 90, 91, 63}  351.82            8  
14  {64, 70, 43, 79, 16, 20, 86, 59, 60, 93}  350.18            5  
18    {5, 70, 39, 8, 74, 43, 14, 79, 17, 60}  349.53           10  

Total de soluciones generadas mediante PR: 4
